# Visualize Commute Routes
This notebook visualizes the commute routes generated by `main.py`. The routes are loaded from the `commute_routes.csv` file and plotted on an interactive map using Plotly.

In [15]:
%pip install pandas plotly geopandas shapely

Note: you may need to restart the kernel to use updated packages.


c:\Users\dexmc\Documents\Code\2025-marketplace-commuteroutes\.venv\Scripts\python.exe: No module named pip


In [16]:
import pandas as pd
import plotly.graph_objects as go
import geopandas as gpd
from shapely import wkt

In [17]:
# Load the data from the CSV file
df = pd.read_csv('commute_routes.csv')

# Filter out rows that do not have a valid LINESTRING in 'line_geometry'
df = df[df['line_geometry'].fillna('').str.startswith('LINESTRING')]

# Convert the 'line_geometry' column from WKT strings to Shapely geometry objects
# and create a GeoDataFrame.
df['geometry'] = df['line_geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="epsg:4326")

print(f"Loaded {len(gdf)} routes after filtering.")
gdf.head()

Loaded 31 routes after filtering.


,origin,destination,travel_mode,distance_km,duration_min,warnings,line_geometry,geometry
0,"St. Albert, AB","Downtown Edmonton, AB",DRIVE,17.96,28.0,NaN,"LINESTRING (-113.62926 53.65391, -113.62816 53...","LINESTRING (-113.62926 53.65391, -113.62816 53..."
1,"Mill Woods, Edmonton","Downtown Edmonton, AB",BICYCLE,14.30,47.0,NaN,"LINESTRING (-113.43435 53.45831, -113.43447 53...","LINESTRING (-113.43435 53.45831, -113.43447 53..."
2,"Eaux Claires Transit Centre, Edmonton","Downtown Edmonton, AB",TRANSIT,11.29,39.0,NaN,"LINESTRING (-113.48736 53.62133, -113.48819 53...","LINESTRING (-113.48736 53.62133, -113.48819 53..."
3,"Larry Uteck, Halifax",Hospital in Halifax,DRIVE,10.25,20.0,NaN,"LINESTRING (-63.67528 44.70207, -63.67451 44.7...","LINESTRING (-63.67528 44.70207, -63.67451 44.7..."
4,"Windsor Street, Halifax","St. Margaret's Bay Road, Halifax",DRIVE,4.94,12.0,NaN,"LINESTRING (-63.60553 44.65555, -63.60534 44.6...","LINESTRING (-63.60553 44.65555, -63.60534 44.6..."


In [18]:
import plotly.graph_objects as go

fig = go.Figure()

# Define a color map for travel modes
travel_mode_colors = {
    'DRIVE': 'blue',
    'TRANSIT': 'red',
    'BICYCLE': 'green',
    'WALK': 'purple'
}
gdf['route_name'] = gdf['origin'] + " to " + gdf['destination']

# Add each route as a separate trace to the map
for i, row in gdf.iterrows():
    # Extract the coordinates from the LineString
    lats = [y for x, y in row.geometry.coords]
    lons = [x for x, y in row.geometry.coords]
    
    fig.add_trace(
        go.Scattermapbox(
            lat=lats,
            lon=lons,
            mode='lines',
            line=dict(width=2, color=travel_mode_colors.get(row['travel_mode'], 'black')),
            name=row['travel_mode'],
            customdata=[row['route_name']],
            hovertemplate='<b>%{customdata[0]}</b><br>Mode: %{name}<extra></extra>',
            visible=True 
        )
    )

# Create updatemenu for routes
buttons = [
    {
        'label': 'All Routes',
        'method': 'restyle',
        'args': ['visible', [True] * len(gdf)]
    }
]

routes = gdf['route_name'].unique()
for route in routes:
    visibility = (gdf['route_name'] == route).tolist()
    buttons.append(
        {
            'label': route,
            'method': 'restyle',
            'args': ['visible', visibility]
        }
    )

# Update the layout to center the map on the routes
fig.update_layout(
    updatemenus=[
        {
            'buttons': buttons,
            'direction': 'down',
            'showactive': True,
            'x': 0.5,
            'xanchor': 'center',
            'y': 1.15,
            'yanchor': 'top'
        }
    ],
    mapbox_style="open-street-map",
    mapbox_bounds={"west": gdf.bounds.minx.min(), "east": gdf.bounds.maxx.max(), "south": gdf.bounds.miny.min(), "north": gdf.bounds.maxy.max()},
    margin={"r":0,"t":40,"l":0,"b":0},
    title="Commute Routes",
    legend_title_text='Travel Mode'
)

fig.show()

C:\Users\dexmc\AppData\Local\Temp\ipykernel_11860\637556214.py:21: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

